# OpenCV

In [23]:
import cv2
import numpy as np
import os
import json
import logging
from tqdm import tqdm


In [24]:
# Configure logging
logging.basicConfig(
    filename='opencv_evaluation.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger()


In [25]:
def calc_iou(box1, box2):
    # Calculate Intersection over Union
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])  # min for intersection
    y2 = min(box1[3], box2[3])  # min for intersection

    intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    union = box1_area + box2_area - intersection

    return intersection / union if union > 0 else 0

def calc_precision(true_positives, false_positives):
    return true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

def calc_recall(true_positives, false_negatives):
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

def calc_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def evaluate_predictions(predictions, ground_truth, iou_threshold=0.5):
    # Evaluate predictions against ground truth
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    matched = [False] * len(ground_truth)

    for pred in predictions:
        matched_any = False
        for i, gt in enumerate(ground_truth):
            if not matched[i] and calc_iou(pred, gt) >= iou_threshold:
                matched[i] = True
                matched_any = True
                true_positives += 1
                break
        if not matched_any:
            false_positives += 1
    false_negatives = len(ground_truth) - sum(matched)

    precision = calc_precision(true_positives, false_positives)
    recall = calc_recall(true_positives, false_negatives)
    f1_score = calc_f1_score(precision, recall)

    return precision, recall, f1_score


In [28]:
def evaluate_opencv_dnn_with_predictions(json_path, output_metrics_path, predictions_output_path, iou_threshold=0.5):
    """
    Evaluate OpenCV DNN face detector on images from the given JSON annotation file,
    and save the model's predictions in a separate JSON file.

    Args:
        json_path (str): Path to the JSON file containing annotations.
        output_metrics_path (str): Path to save the evaluation metrics as a JSON file.
        predictions_output_path (str): Path to save the model predictions as a JSON file.
        iou_threshold (float): IoU threshold to consider a prediction as true positive.

    Returns:
        None
    """
    # Load the annotations
    with open(json_path, 'r') as json_file:
        annotations = json.load(json_file)

    # Initialize the OpenCV DNN face detector
    model_path = r"C:\Users\sotir\Project\models\OpenCV\deploy.prototxt"
    weights_path = r"C:\Users\sotir\Project\models\OpenCV\res10_300x300_ssd_iter_140000_fp16.caffemodel"
    net = cv2.dnn.readNetFromCaffe(model_path, weights_path)
    logger.info("Initialized OpenCV DNN face detector.")

    # Metrics for all images
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Store predictions
    predictions_data = []

    # Iterate over all image annotations with a progress bar
    for index, image_annotation in enumerate(tqdm(annotations, desc="Evaluating Images")):
        # Extract ground truths
        ground_truths = []
        for gt in image_annotation.get('image_info', []):
            bbox = gt.get('bbox', [])
            if len(bbox) != 4:
                logger.warning(f"Invalid bbox format in image {image_annotation.get('image_path', 'unknown')}. Skipping this ground truth.")
                continue
            x, y, w, h = bbox
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            ground_truths.append([x1, y1, x2, y2])

        # Load the image
        image_path = image_annotation.get('image_path', '')
        if not image_path:
            logger.warning(f"No image path provided for annotation index {index}. Skipping.")
            continue

        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Error: Could not load image at {image_path}. Skipping.")
            continue

        (h, w) = image.shape[:2]

        # Preprocess the image for the DNN model
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
                                     (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        predictions = []
        # Iterate over detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                # Get bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                x1, y1, x2, y2 = box.astype("int")
                # Ensure coordinates are within image boundaries
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w - 1, x2), min(h - 1, y2)
                predictions.append([int(x1), int(y1), int(x2), int(y2)])

        # Save predictions for this image
        predictions_data.append({
            "image_path": image_path,
            "predicted_boxes": predictions
        })

        # Calculate metrics
        if not predictions and not ground_truths:
            precision, recall, f1_score = 0, 0, 0
        elif not predictions:
            precision, recall, f1_score = 0, 0, 0
        elif not ground_truths:
            precision, recall, f1_score = 0, 0, 0
        else:
            precision, recall, f1_score = evaluate_predictions(predictions, ground_truths, iou_threshold)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1_score)

    # Calculate mean metrics
    mean_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    mean_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
    mean_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0

    # Log and print the results
    logger.info(f"Mean Precision: {mean_precision:.4f}")
    logger.info(f"Mean Recall: {mean_recall:.4f}")
    logger.info(f"Mean F1-Score: {mean_f1:.4f}")

    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"Mean F1-Score: {mean_f1:.4f}")

    # Save metrics to a JSON file
    output_data = {
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "mean_f1_score": mean_f1,
        "individual_scores": {
            "precision_scores": precision_scores,
            "recall_scores": recall_scores,
            "f1_scores": f1_scores
        }
    }

    try:
        with open(output_metrics_path, 'w') as output_file:
            json.dump(output_data, output_file, indent=4)
        logger.info(f"Metrics saved to {output_metrics_path}")
        print(f"Metrics saved to {output_metrics_path}")
    except Exception as e:
        logger.error(f"Error saving metrics to {output_metrics_path}: {e}")
        print(f"Error saving metrics to {output_metrics_path}: {e}")

    # Save predictions to a separate JSON file
    try:
        with open(predictions_output_path, 'w') as pred_file:
            json.dump(predictions_data, pred_file, indent=4)
        logger.info(f"Predictions saved to {predictions_output_path}")
        print(f"Predictions saved to {predictions_output_path}")
    except Exception as e:
        logger.error(f"Error saving predictions to {predictions_output_path}: {e}")
        print(f"Error saving predictions to {predictions_output_path}: {e}")


In [29]:
json_path = "Data/WIDER FACE Validation Set/wider_face_num_faces_easy.json"
output_path_metrics = "Outputs/OpenCV/metrics_easy_num_faces_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_easy_num_faces_opencv.json"

evaluate_opencv_dnn_with_predictions(json_path, output_path_metrics, output_path_predictions)

Evaluating Images: 100%|███████████████████████████████████████████████████████████| 1122/1122 [00:46<00:00, 24.10it/s]

Mean Precision: 0.7019
Mean Recall: 0.7086
Mean F1-Score: 0.7041
Metrics saved to Outputs/OpenCV/metrics_easy_num_faces_opencv.json
Predictions saved to Outputs/OpenCV/predictions_easy_num_faces_opencv.json


In [30]:
json_path = "Data/WIDER FACE Validation Set/wider_face_num_faces_medium.json"
output_path_metrics = "Outputs/OpenCV/metrics_medium_num_faces_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_medium_num_faces_opencv.json"

evaluate_opencv_dnn_with_predictions(json_path, output_path_metrics, output_path_predictions)

Evaluating Images: 100%|███████████████████████████████████████████████████████████| 1089/1089 [00:46<00:00, 23.55it/s]


Mean Precision: 0.6232
Mean Recall: 0.4518
Mean F1-Score: 0.5002
Metrics saved to Outputs/OpenCV/metrics_medium_num_faces_opencv.json
Predictions saved to Outputs/OpenCV/predictions_medium_num_faces_opencv.json


In [31]:
json_path = "Data/WIDER FACE Validation Set/wider_face_num_faces_hard.json"
output_path_metrics = "Outputs/OpenCV/metrics_hard_num_faces_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_hard_num_faces_opencv.json"

evaluate_opencv_dnn_with_predictions(json_path, output_path_metrics, output_path_predictions)

Evaluating Images: 100%|███████████████████████████████████████████████████████████| 1015/1015 [00:43<00:00, 23.12it/s]

Mean Precision: 0.3421
Mean Recall: 0.0814
Mean F1-Score: 0.1154
Metrics saved to Outputs/OpenCV/metrics_hard_num_faces_opencv.json
Predictions saved to Outputs/OpenCV/predictions_hard_num_faces_opencv.json


In [34]:
import random
def evaluate_opencv_dnn_with_random_subset(json_path, output_metrics_path, predictions_output_path, iou_threshold=0.5,num_samples=None, seed=42):
    """
    Evaluate OpenCV DNN face detector on images from the given JSON annotation file,
    and save the model's predictions in a separate JSON file.

    Args:
        json_path (str): Path to the JSON file containing annotations.
        output_metrics_path (str): Path to save the evaluation metrics as a JSON file.
        predictions_output_path (str): Path to save the model predictions as a JSON file.
        iou_threshold (float): IoU threshold to consider a prediction as true positive.

    Returns:
        None
    """
    # Load the annotations
    with open(json_path, 'r') as json_file:
        annotations = json.load(json_file)

    # Randomly sample the dataset if num_samples is specified
    if num_samples is not None:
        random.seed(seed)
        sampled_size = min(num_samples, len(annotations))
        annotations = random.sample(annotations, sampled_size)
        logger.info(f"Randomly sampled {sampled_size} annotations with seed {seed}.")

    # Initialize the OpenCV DNN face detector
    model_path = r"C:\Users\sotir\Project\models\OpenCV\deploy.prototxt"
    weights_path = r"C:\Users\sotir\Project\models\OpenCV\res10_300x300_ssd_iter_140000_fp16.caffemodel"
    net = cv2.dnn.readNetFromCaffe(model_path, weights_path)
    logger.info("Initialized OpenCV DNN face detector.")

    # Metrics for all images
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Store predictions
    predictions_data = []

    # Iterate over all image annotations with a progress bar
    for index, image_annotation in enumerate(tqdm(annotations, desc="Evaluating Images")):
        # Extract ground truths
        ground_truths = []
        for gt in image_annotation.get('image_info', []):
            bbox = gt.get('bbox', [])
            if len(bbox) != 4:
                logger.warning(f"Invalid bbox format in image {image_annotation.get('image_path', 'unknown')}. Skipping this ground truth.")
                continue
            x, y, w, h = bbox
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            ground_truths.append([x1, y1, x2, y2])

        # Load the image
        image_path = image_annotation.get('image_path', '')
        if not image_path:
            logger.warning(f"No image path provided for annotation index {index}. Skipping.")
            continue

        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Error: Could not load image at {image_path}. Skipping.")
            continue

        (h, w) = image.shape[:2]

        # Preprocess the image for the DNN model
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
                                     (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        predictions = []
        # Iterate over detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                # Get bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                x1, y1, x2, y2 = box.astype("int")
                # Ensure coordinates are within image boundaries
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w - 1, x2), min(h - 1, y2)
                predictions.append([int(x1), int(y1), int(x2), int(y2)])

        # Save predictions for this image
        predictions_data.append({
            "image_path": image_path,
            "predicted_boxes": predictions
        })

        # Calculate metrics
        if not predictions and not ground_truths:
            precision, recall, f1_score = 0, 0, 0
        elif not predictions:
            precision, recall, f1_score = 0, 0, 0
        elif not ground_truths:
            precision, recall, f1_score = 0, 0, 0
        else:
            precision, recall, f1_score = evaluate_predictions(predictions, ground_truths, iou_threshold)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1_score)

    # Calculate mean metrics
    mean_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    mean_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
    mean_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0

    # Log and print the results
    logger.info(f"Mean Precision: {mean_precision:.4f}")
    logger.info(f"Mean Recall: {mean_recall:.4f}")
    logger.info(f"Mean F1-Score: {mean_f1:.4f}")

    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"Mean F1-Score: {mean_f1:.4f}")

    # Save metrics to a JSON file
    output_data = {
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "mean_f1_score": mean_f1,
        "individual_scores": {
            "precision_scores": precision_scores,
            "recall_scores": recall_scores,
            "f1_scores": f1_scores
        }
    }

    try:
        with open(output_metrics_path, 'w') as output_file:
            json.dump(output_data, output_file, indent=4)
        logger.info(f"Metrics saved to {output_metrics_path}")
        print(f"Metrics saved to {output_metrics_path}")
    except Exception as e:
        logger.error(f"Error saving metrics to {output_metrics_path}: {e}")
        print(f"Error saving metrics to {output_metrics_path}: {e}")

    # Save predictions to a separate JSON file
    try:
        with open(predictions_output_path, 'w') as pred_file:
            json.dump(predictions_data, pred_file, indent=4)
        logger.info(f"Predictions saved to {predictions_output_path}")
        print(f"Predictions saved to {predictions_output_path}")
    except Exception as e:
        logger.error(f"Error saving predictions to {predictions_output_path}: {e}")
        print(f"Error saving predictions to {predictions_output_path}: {e}")

# Testing on Blur

### Easy blur

In [36]:
json_path = "Data/WIDER FACE Validation Set/wider_face_easy_blur.json"
output_path_metrics = "Outputs/OpenCV/metrics_easy_blur_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_easy_blur_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=700)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 700/700 [00:28<00:00, 24.19it/s]

Mean Precision: 0.8439
Mean Recall: 0.7988
Mean F1-Score: 0.8099
Metrics saved to Outputs/OpenCV/metrics_easy_blur_opencv.json
Predictions saved to Outputs/OpenCV/predictions_easy_blur_opencv.json


### Medium blur

In [37]:
json_path = "Data/WIDER FACE Validation Set/wider_face_medium_blur.json"
output_path_metrics = "Outputs/OpenCV/metrics_medium_blur_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_medium_blur_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=700)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 700/700 [00:27<00:00, 25.08it/s]

Mean Precision: 0.4996
Mean Recall: 0.3252
Mean F1-Score: 0.3699
Metrics saved to Outputs/OpenCV/metrics_medium_blur_opencv.json
Predictions saved to Outputs/OpenCV/predictions_medium_blur_opencv.json


### Hard Blur

In [38]:
json_path = "Data/WIDER FACE Validation Set/wider_face_hard_blur.json"
output_path_metrics = "Outputs/OpenCV/metrics_hard_blur_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_hard_blur_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=700)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 700/700 [00:27<00:00, 25.56it/s]

Mean Precision: 0.2779
Mean Recall: 0.0642
Mean F1-Score: 0.0933
Metrics saved to Outputs/OpenCV/metrics_hard_blur_opencv.json
Predictions saved to Outputs/OpenCV/predictions_hard_blur_opencv.json


# Testing on occlusion

### Easy occlusion

In [39]:
json_path = "Data/WIDER FACE Validation Set/wider_face_easy_occlusion.json"
output_path_metrics = "Outputs/OpenCV/metrics_easy_occlusion_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_easy_occlusion_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=300)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 24.12it/s]

Mean Precision: 0.6883
Mean Recall: 0.6248
Mean F1-Score: 0.6426
Metrics saved to Outputs/OpenCV/metrics_easy_occlusion_opencv.json
Predictions saved to Outputs/OpenCV/predictions_easy_occlusion_opencv.json


### Medium occlusion

In [40]:
json_path = "Data/WIDER FACE Validation Set/wider_face_medium_occlusion.json"
output_path_metrics = "Outputs/OpenCV/metrics_medium_occlusion_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_medium_occlusion_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=300)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 24.32it/s]

Mean Precision: 0.4353
Mean Recall: 0.2473
Mean F1-Score: 0.2870
Metrics saved to Outputs/OpenCV/metrics_medium_occlusion_opencv.json
Predictions saved to Outputs/OpenCV/predictions_medium_occlusion_opencv.json


### Hard occlusion

In [41]:
json_path = "Data/WIDER FACE Validation Set/wider_face_hard_occlusion.json"
output_path_metrics = "Outputs/OpenCV/metrics_hard_occlusion_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_hard_occlusion_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=300)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 24.37it/s]

Mean Precision: 0.3649
Mean Recall: 0.1058
Mean F1-Score: 0.1434
Metrics saved to Outputs/OpenCV/metrics_hard_occlusion_opencv.json
Predictions saved to Outputs/OpenCV/predictions_hard_occlusion_opencv.json


# Testing on illumination

### Normal illumination

In [42]:
json_path = "Data/WIDER FACE Validation Set/wider_face_normal_illumination.json"
output_path_metrics = "Outputs/OpenCV/metrics_normal_illumination_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_normal_illumination_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=400)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 400/400 [00:16<00:00, 24.59it/s]

Mean Precision: 0.6325
Mean Recall: 0.5043
Mean F1-Score: 0.5268
Metrics saved to Outputs/OpenCV/metrics_normal_illumination_opencv.json
Predictions saved to Outputs/OpenCV/predictions_normal_illumination_opencv.json


### Extreme illumination

In [43]:
json_path = "Data/WIDER FACE Validation Set/wider_face_extreme_illumination.json"
output_path_metrics = "Outputs/OpenCV/metrics_extreme_illumination_opencv.json"
output_path_predictions = "Outputs/OpenCV/predictions_extreme_illumination_opencv.json"

evaluate_opencv_dnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=400)

Evaluating Images: 100%|█████████████████████████████████████████████████████████████| 400/400 [00:16<00:00, 24.89it/s]

Mean Precision: 0.4208
Mean Recall: 0.2355
Mean F1-Score: 0.2634
Metrics saved to Outputs/OpenCV/metrics_extreme_illumination_opencv.json
Predictions saved to Outputs/OpenCV/predictions_extreme_illumination_opencv.json


In [45]:
def visualize_detections(image_path, predictions, ground_truths=None):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not load image {image_path}")
        return
    # Draw predictions in red
    for pred in predictions:
        x1, y1, x2, y2 = pred
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    # Draw ground truths in green
    if ground_truths:
        for gt in ground_truths:
            x1, y1, x2, y2 = gt
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # Show image
    cv2.imshow("Detections", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [52]:
# Load the predictions and annotations
with open(output_path_predictions, 'r') as pred_file:
    predictions_data = json.load(pred_file)

with open(json_path, 'r') as json_file:
    annotations = json.load(json_file)

# Choose an image to visualize
sample_index = 150  # Change as needed
image_path = predictions_data[sample_index]["image_path"]
predictions = predictions_data[sample_index]["predicted_boxes"]

# Find ground truths for the image
ground_truths = []
for ann in annotations:
    if ann["image_path"] == image_path:
        for gt in ann.get("image_info", []):
            bbox = gt.get('bbox', [])
            x, y, w, h = bbox
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            ground_truths.append([x1, y1, x2, y2])
        break

visualize_detections(image_path, predictions, ground_truths)


In [54]:
import time
def calculate_average_prediction_time_opencv_dnn(json_path, prototxt_path, model_path, confidence_threshold=0.5, max_images=100, seed=42):
    """
    Calculate the average prediction time of OpenCV's DNN-based face detector on a random subset of images.

    Args:
        json_path (str): Path to the JSON file containing annotations.
        prototxt_path (str): Path to the DNN model's prototxt file.
        model_path (str): Path to the DNN model's weights file.
        confidence_threshold (float): Minimum confidence threshold to filter weak detections.
        max_images (int): Number of images to process.
        seed (int): Random seed for reproducibility.

    Returns:
        float: The average prediction time per image in seconds.
    """
    # Check if the JSON file exists
    if not os.path.exists(json_path):
        logger.error(f"JSON file does not exist: {json_path}")
        print(f"Error: JSON file does not exist: {json_path}")
        return None

    # Load the annotations
    try:
        with open(json_path, 'r') as json_file:
            annotations = json.load(json_file)
            logger.info(f"Loaded {len(annotations)} annotations from {json_path}")
    except json.JSONDecodeError as e:
        logger.error(f"Error decoding JSON file: {e}")
        print(f"Error decoding JSON file: {e}")
        return None

    # Randomly select a subset of the data
    random.seed(seed)
    if len(annotations) > max_images:
        annotations_subset = random.sample(annotations, max_images)
        logger.info(f"Selected {max_images} random images for evaluation.")
    else:
        annotations_subset = annotations
        logger.warning(f"Dataset contains fewer than {max_images} images. Using all available images ({len(annotations_subset)}).")

    # Initialize the DNN-based face detector
    if not os.path.exists(prototxt_path):
        logger.error(f"Prototxt file does not exist: {prototxt_path}")
        print(f"Error: Prototxt file does not exist: {prototxt_path}")
        return None

    if not os.path.exists(model_path):
        logger.error(f"Caffe model file does not exist: {model_path}")
        print(f"Error: Caffe model file does not exist: {model_path}")
        return None

    try:
        net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
        logger.info("Initialized OpenCV DNN-based face detector.")
    except Exception as e:
        logger.error(f"Error loading DNN model: {e}")
        print(f"Error loading DNN model: {e}")
        return None

    # Measure prediction times
    prediction_times = []

    for image_annotation in tqdm(annotations_subset, desc="Measuring Prediction Times"):
        # Load the image
        image_path = image_annotation.get('image_path', '')
        if not image_path:
            logger.warning("No image path provided for an annotation. Skipping.")
            continue

        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}. Skipping.")
            print(f"Error: Image file does not exist: {image_path}. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Error: Could not load image at {image_path}. Skipping.")
            print(f"Error: Could not load image at {image_path}. Skipping.")
            continue

        (h, w) = image.shape[:2]

        # Prepare the blob for DNN
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
                                     (300, 300), (104.0, 177.0, 123.0))

        # Start the timer
        start_time = time.time()

        # Set the blob as input to the network and perform a forward-pass to get detections
        net.setInput(blob)
        detections = net.forward()

        # End the timer
        end_time = time.time()

        # Calculate prediction time
        prediction_time = end_time - start_time
        prediction_times.append(prediction_time)

    # Calculate the average prediction time
    if prediction_times:
        average_time = sum(prediction_times) / len(prediction_times)
        logger.info(f"Average prediction time (OpenCV DNN): {average_time:.4f} seconds per image")
        print(f"Average prediction time (OpenCV DNN): {average_time:.4f} seconds per image")
        return average_time
    else:
        logger.warning("No valid images were processed.")
        print("No valid images were processed.")
        return None

# Example usage
if __name__ == "__main__":
    json_path = "Data/WIDER FACE Validation Set/wider_face_val_annotations.json"
    prototxt_path = r"C:\Users\sotir\Project\models\OpenCV\deploy.prototxt"  # Update with your path
    model_path = r"C:\Users\sotir\Project\models\OpenCV\res10_300x300_ssd_iter_140000_fp16.caffemodel"
    calculate_average_prediction_time_opencv_dnn(json_path, prototxt_path, model_path, confidence_threshold=0.5, max_images=100, seed=42)




Measuring Prediction Times: 100%|████████████████████████████████████████████████████| 100/100 [00:03<00:00, 26.70it/s]

Average prediction time (OpenCV DNN): 0.0282 seconds per image
